# Data Challenge 5: Bringing It All Together

## Rocio Jaime, Sophie Li, Ester Zhao

### St. Himark Timeline of Events

In [81]:
# help

In [82]:
# imports
import geopandas as gpd
import scipy.stats as st

import datetime
import base64
import random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

### Suggested Resource Allocation Using Rumble App Data

   The Rumble app begins to receive a small wave of damage reports starting on April 6,2020 at 16:00 briefly after the initial shock. A substantial surge of Rumble reports follows on April 8th at 9:00am, primarily in Old Town and the southern neighborhoods closer to the coast (Broadview, Chapparal,and Scenic Vista). Old Town sent the most reports with the highest occurring category being power. Because there were already efforts to modernize the electrical distribution system, it is possible that the outdated infrastructure was especially vulnerable to the earthquake. The Old Town area was also nearest to the epicenter of the earthquake’s aftershock. The southern neighborhoods had a greater number of reports pertaining to water and sewage.

   We see a second surge of reports on the afternoon of April 9th, most likely due to a power outage which caused a delay of report receipts. It is possible that many of these reports were sent in between April 9th 3:00am to April 9th 12:00. During this window there were little to no reports sent from nearly all neighborhoods in St. Himark. Based on Rumble data alone, we’d recommend sending emergency services to Old Town(3), Pepper Mill (12), and the southernmost neighborhoods -- Broadview, Chaparral, and Scenic Vista -- due to the significantly higher volume of Rumble reports. 


In [83]:
# load in csv
damage_reports = pd.read_csv("https://raw.githubusercontent.com/data-challengers/DC5/main/DC5-Data/Damage%20Reports/mc1-reports-data.csv")

# convert timevariable to datetime
damage_reports['time'] = pd.to_datetime(damage_reports['time'])
# round to hour for scroll
damage_reports['time'] = damage_reports['time'].round('H')
damage_reports.fillna(0, inplace = True)

# make data long so we can group by and facet plot

damage_reports_long = pd.melt(damage_reports, id_vars=['time', 'location'], var_name = 'category', value_name = 'value')

damage_summed = damage_reports_long.groupby(['location', 'time', 'category'], as_index = False).agg({'value' : 'sum'})

In [84]:
fig = px.line(damage_summed, 
              x="time", 
              y="value", 
              color="category", 
              facet_col="location",
              facet_col_wrap=4, 
              height=1300, 
              width=1000,
              facet_row_spacing=.02,
             title="Total Rumble Reports Over Time",
             labels=dict(time="Time", value="Count"))

fig.show()

### Y*Int Social Media Data and Possible Usage

   Many citizens of St. Himark use the social media app Y\*int to communicate with their community. We decided to look into whether Y\*int could be used as a viable method for city officials to survey the wellbeing of the population, and potentially anticipate needs. In particular, we wanted to see if Y\*int survelliance could be used as an alternative when the Rumble app experienced outages due to power loss.
   
   This initial graph displays the Y\*int messages by neighborhood, and show a large spike midday on April 8th, when the largest and most disruptive earthquake event occurred. 


In [85]:
def find_string_in_messages(df, string_to_find):
    """
    Find a string in df and return a subset df that of messages that include the string
    :param: df: dataframe of interest
    :param: string_to_find: the string to find in message
    """
    subset_df = df[df['message'].str.contains(string_to_find, na=False, case=False)]
    subset_df.reset_index(inplace=True)
    return subset_df

def get_account_message_counts(df):
    """
    For each account in the dataframe, creates a count for how many messages they have sent
    and returns a dataframe that contains account names and count.
    :param: df: dataframe of interest
    """
    account_sums = df.groupby(['account']).size().reset_index(name='count')
    account_sums = account_sums.sort_values(by=['count'], ascending=False)
    return account_sums

def get_topic_df(df, keywords):
    """
    Creates a subset df of messages that include strings from a list of keywords
    :param: df: dataframe of interest
    :param: keywords: list of strings to select for
    """
    mask = df[df['message'].str.contains('|'.join(keywords), na=False, case=False)]
    mask.reset_index(inplace=True, drop=True)
    return mask

def fill_df_nas(df, time_col, group_col, group_arr):
    """
    Expands dataframe to include all x-axis values for every group, and
    fills dataframes with NAs when there are no observations for the specified group.
    
    Modified function that includes NA values for all time points within range from min-max times observed.
    
    Useful for Plotly graphs in mode='lines+markers'
    :param: df: dataframe of interest
    :param: time_col: string name of column that contains time variable (or generally, the x variable)
    :param: group_col: string name of column that contains the groups to plot over different traces
    :param: group_arr: list or numpy array of all unique observations in df['group_col']
    """
    # Sort by time for graphing
    df = df.sort_values(by=[time_col])
    # Get series time range between min and max time points
    time_range = pd.date_range(df[time_col].min(), df[time_col].max(), freq='H')
    
    df_series = pd.Series(np.tile(group_arr, len(time_range)))
    df_idx_series = time_range \
        .repeat(len(group_arr))
    new_df = pd.DataFrame({time_col: df_idx_series,
                          group_col: df_series})
    df_with_nas = pd.merge(new_df, df, on=[time_col, group_col], how='left')
    return df_with_nas

# Round times to the hour (to be grouped for plotting)
def create_message_counts(df):
    df['hour'] = df['time'].dt.floor('h')
    # Create count aggregate of classification appearances over time
    grouped_df = df.groupby(['location', 'hour']).size().reset_index(name='count')
    return grouped_df

def clean_yint_data(file_url):
    """
    Takes in a hosted csv file of yint messages and returns a cleaned dataframe
    Removes standard bots to allow gov officials to quickly remove non-community member posts
    
    Removes commercial bots promoting deals and bots promoting usage of the rumble app
    
    Note: recommend doing spot checks to ensure real users are not being removed
    
    :param: file_url: the hosted location of a raw csv file containing y*int data
    """
    df = pd.read_csv(file_url, parse_dates=[0], infer_datetime_format=True)
    
    # Create time frame for data
    start_date = min(df['time'])
    end_date = max(df['time'])
    delta = end_date - start_date
    days = delta.days
    # get a standard amount of posts per day that would probably be a bot
    # in this case, I set the standard to 10+ posts a day
    bot_level_posting = 10*days 
    
    # Eliminate "deals" bots promoting commercial info
    bot_keywords = ['deal', 'sale', 'rumble']
    potential_bot_df = get_topic_df(df, bot_keywords)
    # Get list of accounts that mention deals
    potential_bot_accounts = potential_bot_df['account'].unique()
    
    # Remove users that don't seem like bots
    boolean_series = df['account'].isin(potential_bot_accounts)
    potential_bot_df = df[boolean_series]
    account_sums = get_account_message_counts(potential_bot_df)
    # NOTE: Recommend doing spot checks every so often to confirm you are not removing real users
    bots = account_sums.loc[account_sums['count'] > bot_level_posting]
    # Get list of bot account names
    bot_accounts = bots['account'].unique()


    # Create a df that doesn't include bots
    cleaned_df = df[~df['account'].isin(bot_accounts)]
    cleaned_df.reset_index(inplace=True)
    return cleaned_df

url = 'https://raw.githubusercontent.com/data-challengers/DC5/main/DC5-Data/YInt%20Social%20Media%20Data/YInt.csv'
df = clean_yint_data(url)

# Get list of neighborhoods being used in Yint dataset
neighborhoods = df['location'].unique()

In [86]:
# Plot frequency of messaging from neighborhoods over time
def plot_messages_over_time(df, title):
    fig = go.Figure()
    # plot all classifications as different lines
    for i in range(0, len(neighborhoods)):
        # filter for specific classification
        expr = df['location'] == neighborhoods[i]
        fig.add_trace(go.Scatter(
            x=df[expr]['hour'],
            y=df[expr]['count'],
            name=neighborhoods[i],
            connectgaps=False,
            mode='lines')
        )
        
    fig.update_layout(
        title_text=title,
        xaxis_title="Time",
        yaxis_title="Count",
        legend_title="Neighborhood")
    
    fig.show()

# Create plot
grouped_df = create_message_counts(df)
fig_df = fill_df_nas(grouped_df, 'hour', 'location', neighborhoods)
plot_messages_over_time(fig_df, 'Y*Int Messages By Neighborhood')

   In order to make raw Y\*int data useful, we created functions that clean the data to remove most bots and business accounts that are not relevant. The file containing these functions can be found at [clean_yint_data.ipynb](https://github.com/data-challengers/DC5/blob/main/notebooks/clean_yint_data.ipynb).
   
   Using this function, officials can pass in new Y\*int data files to receive updated graphs that display messages related to earthquakes or disasters. For example, the graph below displays the earthquake-related messages from the week of April 6th by neighborhood.

In [87]:
def create_neighborhood_grouped_count_plot(df, title):
    grouped_df = create_message_counts(df)
    fig_df = fill_df_nas(grouped_df, 'hour', 'location', neighborhoods)
    plot_messages_over_time(fig_df, title)


alert_topics = ['quake', 'quaking', 'aftershock', 'shake', 'shaking', 'outage', 'sewage', 'fire']
alert_df = get_topic_df(df, alert_topics)
alert_df.to_csv('additional_data/yint_alerts.csv', index=False) # save alerts csv 
create_neighborhood_grouped_count_plot(alert_df, 'Y*Int Earthquake-Related Messages')

In order to evaluate the usefulness of Y\*Int data as a potential alternative to the Rumble app, we mapped Rumble alerts and Y\*int earthquake-related messages together. At midday on April 8th, we see a sharp decline in Rumble alerts, which we inferred was due to a mass power outage. Although Y\*int data also experienced a serious of drops during that time frame, it was to a lesser extent than the Rumble app. As such, we conclude that while the Rumble app will be more useful for government officials to use on a regular basis for allocating resources, Y\*int data created using our functions can be considered a feasible backup option.

In [88]:
url = 'https://raw.githubusercontent.com/data-challengers/DC5/main/DC5-Data/Damage%20Reports/mc1-reports-data.csv'
rumble_df = pd.read_csv(url, parse_dates=[0], infer_datetime_format=True)
rumble_df.head()

# Round times to the hour (to be grouped for plotting)
def create_hour_aggregated_df(df):
    df['hour'] = df['time'].dt.floor('h')
    # Create count aggregate of classification appearances over time
    grouped_df = df.groupby('hour').size().reset_index(name='count')
    return grouped_df


rumble_df_agg = create_hour_aggregated_df(rumble_df)
yint_df_agg = create_hour_aggregated_df(alert_df)

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# plot rumble messages
fig.add_trace(go.Scatter(
    x=rumble_df_agg['hour'],
    y=rumble_df_agg['count'],
    name='Rumble',
    connectgaps=False,
    mode='lines'),
    secondary_y=False
)
# plot yint earthquake messages
fig.add_trace(go.Scatter(
    x=yint_df_agg['hour'],
    y=yint_df_agg['count'],
    name='Y*int',
    connectgaps=False,
    mode='lines'),
    secondary_y=True
)

# Set y-axes titles
fig.update_yaxes(title_text="Rumble message count", secondary_y=False)
fig.update_yaxes(title_text="Y*int message count", secondary_y=True)


fig.update_layout(
    title_text='Y*int Earthquake-Related Messages vs Rumble alerts',
    xaxis_title="Time",
    legend_title="Media")

fig.show()



Finally, we also used Y\*int data to create a more comprehensive timeline of the events that occured over the past few days, for future reference. To do so, we pulled all messages sent by @AlwaysSafePowerCompany, @DOT-StHimark, and @DeptTransport.

### Sensor Data Analysis

We examined the mobile and static sensor data in order to find potential radiation patterns. We plotted the mean radiation level per hour, and grouped each sensor by neighborhood. The Cheddarford, Palace Hills, and Downtown neighborhoods show moderately increasing levels of radiation starting on the evening of April 9th, while the Southwest neighborhood shows a more abrupt increase in counts per minute in the same time period.

In [89]:
data_path = "DC5-Data/"
static_data = pd.read_csv(data_path + "Sensor Data and Maps/StaticSensorReadings.csv", parse_dates=[0], infer_datetime_format=True)

In [90]:
fp = data_path + "Sensor Data and Maps/StHimarkNeighborhoodShapefiles/StHimark.shp"
#reading the file stored in variable fp
map_df = gpd.read_file(fp)
map_df = map_df.to_crs(epsg=4326)

In [91]:
def assign_neighborhood(x, map_df, geo_col, name_col):
    expr = map_df[geo_col].contains(x)
    return map_df[expr][name_col].values[0] # return first neighborhood

sensor_locations = pd.read_csv(data_path + "Sensor Data and Maps/StaticSensorLocations.csv")
sensor_locations_gdf = gpd.GeoDataFrame(
    sensor_locations, geometry=gpd.points_from_xy(sensor_locations.Long, sensor_locations.Lat))
sensor_locations_gdf['neighborhood'] = sensor_locations_gdf['geometry'].apply(
    assign_neighborhood, args=(map_df, 'geometry', 'Nbrhood',))

subplot_idx = pd.DataFrame({'neighborhood': sensor_locations_gdf['neighborhood'].unique()})
subplot_idx['row'] = (subplot_idx.index // 2) + 1
subplot_idx['col'] = (subplot_idx.index % 2) + 1

static_data['time_round'] = static_data['Timestamp'].dt.floor('h')

In [92]:
def conf_int(x):
    interval = st.t.interval(alpha=0.95, df=len(x)-1, loc=np.mean(x), scale=st.sem(x)) 
    return interval

static_data_grouped = static_data.groupby(['time_round', 'Sensor-id'])
static_data_mean = static_data_grouped.mean()
static_data_mean['lower'] = static_data_grouped.apply(lambda x: conf_int(x['Value'].to_numpy())[0])
static_data_mean['upper'] = static_data_grouped.apply(lambda x: conf_int(x['Value'].to_numpy())[1])

static_sensors = static_data['Sensor-id'].unique()

min_time = static_data_mean.index.get_level_values(0).min()
max_time = static_data_mean.index.get_level_values(0).max()
time_range = pd.date_range(min_time, max_time, freq='H')
df_series = pd.Series(np.tile(static_sensors, len(time_range)))
df_idx_series = time_range \
        .repeat(len(static_sensors))
new_df = pd.DataFrame({'time_round': df_idx_series,
                          'Sensor-id': df_series})
new_df.set_index(['time_round', 'Sensor-id'], inplace=True)

df_with_nas = pd.DataFrame.join(new_df, static_data_mean, how='left')

static_sensors.sort()

df_with_nas = pd.merge(df_with_nas.reset_index(), sensor_locations, on='Sensor-id')
df_with_nas.set_index(['time_round', 'Sensor-id'], inplace=True)

In [93]:
fig = make_subplots(rows=4, cols=2,
                   subplot_titles=subplot_idx['neighborhood'].to_numpy())
for i in range(len(static_sensors)):
    expr = df_with_nas.index.get_level_values(1) == static_sensors[i]
    xval = df_with_nas[expr].index.get_level_values(0).to_series()
    nexpr = subplot_idx['neighborhood'] == sensor_locations_gdf[sensor_locations_gdf['Sensor-id']==static_sensors[i]]['neighborhood'].item()
    fig.add_trace(go.Scatter(
        x=xval,
        y=df_with_nas[expr]['Value'],
        mode='lines',
        connectgaps=False,
        name=str(static_sensors[i])
    ),
                 row=subplot_idx[nexpr]['row'].item(), col=subplot_idx[nexpr]['col'].item())
    fig.add_trace(go.Scatter(
        x=pd.concat((xval, xval[::-1])), # x, then x reversed
        y=pd.concat((df_with_nas[expr]['lower'], df_with_nas[expr]['upper'][::-1])), # upper, then lower reversed
        fill='toself',
        fillcolor='rgba(0,100,80,0.2)',
        line=dict(color='rgba(255,255,255,0)'),
        hoverinfo="skip",
        connectgaps=True,
        mode='lines',
        showlegend=False
    ),
                 row=subplot_idx[nexpr]['row'].item(), col=subplot_idx[nexpr]['col'].item())

fig.update_layout(
    height=600, 
    title_text='Raditation Data by Neighborhood in Counts per Minute', 
    legend_title='Sensor', yaxis_title='counts per minute')

fig.show()

# Sophie inserts the rest of her stuff here!

### Next Steps and Conclusions

The current situation in St. Himark is messy, and the unreliability of our different data streams made it difficult to create 

Given more time, our team would have liked to look into creating neighborhood profiles in order to gauge reliability of different reports. This would allow city officials to quickly gauge which neighborhoods were probably experiences power outages, and which truly had the highest priority for resource allocation.

Furthermore, we would have liked to do additional analysis on the ways that the Y\*int social media network could be synthesized with other data streams to create a dynamic analysis of city conditions. A more involved online awareness would also allow the city to communicate more openly with citizens, which could boost morale and calm hysteria in case of further emergencies.